In [ ]:
# Guardar resultados comparativos finales
if COMPARISON_ENABLED and ada_embeddings is not None:
    print("💾 Guardando resultados comparativos...")
    
    # Compilar todos los resultados
    comparison_results = {
        'metadata': {
            'timestamp': datetime.now().isoformat(),
            'ada_model': 'text-embedding-ada-002',
            'e5_model': 'intfloat/e5-large-v2',
            'total_documents_compared': len(ada_embeddings),
            'colab_gpu': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU',
            'analysis_sample_size': similarity_metrics['sample_size']
        },
        'ada_properties': ada_properties,
        'e5_properties': e5_properties,
        'similarity_metrics': similarity_metrics,
        'interpretation': {
            'concordance_level': 'alta' if similarity_metrics['cosine_similarity_mean'] > 0.8 else 
                               'moderada' if similarity_metrics['cosine_similarity_mean'] > 0.6 else 'baja',
            'correlation_strength': 'fuerte' if similarity_metrics['pearson_correlation'] > 0.7 else
                                  'moderada' if similarity_metrics['pearson_correlation'] > 0.4 else 'débil',
            'recommendation': 'Los modelos muestran comportamiento similar' if similarity_metrics['cosine_similarity_mean'] > 0.7 
                            else 'Los modelos capturan aspectos diferentes del texto'
        }
    }
    
    # Guardar resultados comparativos
    comparison_file = f"{DRIVE_BASE}/ada_vs_e5_comparison_results.json"
    with open(comparison_file, 'w', encoding='utf-8') as f:
        json.dump(comparison_results, f, ensure_ascii=False, indent=2)
    
    print(f"✅ Resultados comparativos guardados en: {comparison_file}")
    
    # Resumen ejecutivo
    print(f"\n📋 RESUMEN EJECUTIVO:")
    print("=" * 50)
    print(f"🎯 Concordancia entre modelos: {comparison_results['interpretation']['concordance_level'].upper()}")
    print(f"🔗 Correlación: {comparison_results['interpretation']['correlation_strength'].upper()}")
    print(f"💡 Recomendación: {comparison_results['interpretation']['recommendation']}")
    
    print(f"\n📊 Métricas clave:")
    print(f"   • Similitud coseno promedio: {similarity_metrics['cosine_similarity_mean']:.4f}")
    print(f"   • Correlación Pearson: {similarity_metrics['pearson_correlation']:.4f}")
    print(f"   • Correlación matrices similitud: {similarity_metrics['similarity_matrix_correlation']:.4f}")
    
    print(f"\n💾 Archivos generados para descarga:")
    print(f"   • {json_output} (embeddings E5)")
    print(f"   • {comparison_file} (análisis comparativo)")
    if 'viz_path' in locals():
        print(f"   • {viz_path} (visualizaciones)")
    
    print(f"\n🎉 ¡Análisis comparativo completado exitosamente!")
    
else:
    print("📝 Resultados comparativos no generados (comparación no habilitada)")
    print("💡 Para generar comparación, sube un archivo con embeddings Ada-002")

In [ ]:
# Generar visualizaciones comparativas (opcional)
if COMPARISON_ENABLED and ada_embeddings is not None:
    try:
        import matplotlib.pyplot as plt
        import seaborn as sns
        
        print("📊 Generando visualizaciones...")
        
        # Configurar estilo
        plt.style.use('default')
        sns.set_palette("husl")
        
        # Crear figura con subplots
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        fig.suptitle('Comparación Ada-002 vs E5-Large-v2', fontsize=16, fontweight='bold')
        
        # 1. Distribución de normas
        ada_norms = np.linalg.norm(ada_embeddings, axis=1)
        e5_norms = np.linalg.norm(e5_embeddings, axis=1)
        
        axes[0,0].hist(ada_norms, bins=50, alpha=0.7, label='Ada-002', density=True)
        axes[0,0].hist(e5_norms, bins=50, alpha=0.7, label='E5-Large-v2', density=True)
        axes[0,0].set_title('Distribución de Normas de Embeddings')
        axes[0,0].set_xlabel('Norma L2')
        axes[0,0].set_ylabel('Densidad')
        axes[0,0].legend()
        axes[0,0].grid(True, alpha=0.3)
        
        # 2. Similitud coseno por documento
        cos_sims = similarity_metrics['cosine_similarity_mean']
        sample_indices = np.random.choice(len(ada_embeddings), min(500, len(ada_embeddings)), replace=False)
        sample_cos_sims = []
        
        for i in sample_indices:
            ada_norm = ada_embeddings[i] / np.linalg.norm(ada_embeddings[i])
            e5_norm = e5_embeddings[i] / np.linalg.norm(e5_embeddings[i])
            cos_sim = np.dot(ada_norm, e5_norm)
            sample_cos_sims.append(cos_sim)
        
        axes[0,1].hist(sample_cos_sims, bins=30, alpha=0.7, color='green')
        axes[0,1].axvline(np.mean(sample_cos_sims), color='red', linestyle='--', 
                         label=f'Media: {np.mean(sample_cos_sims):.3f}')
        axes[0,1].set_title('Distribución de Similitud Coseno')
        axes[0,1].set_xlabel('Similitud Coseno')
        axes[0,1].set_ylabel('Frecuencia')
        axes[0,1].legend()
        axes[0,1].grid(True, alpha=0.3)
        
        # 3. Comparación de dimensiones (muestra)
        sample_dims = min(20, ada_embeddings.shape[1])
        dim_indices = np.random.choice(ada_embeddings.shape[1], sample_dims, replace=False)
        
        ada_means = [np.mean(ada_embeddings[:, dim]) for dim in dim_indices]
        e5_means = [np.mean(e5_embeddings[:, dim]) for dim in dim_indices]
        
        axes[1,0].scatter(ada_means, e5_means, alpha=0.6)
        axes[1,0].plot([min(ada_means + e5_means), max(ada_means + e5_means)], 
                      [min(ada_means + e5_means), max(ada_means + e5_means)], 
                      'r--', alpha=0.8, label='y=x')
        axes[1,0].set_title('Correlación de Medias por Dimensión')
        axes[1,0].set_xlabel('Media Ada-002')
        axes[1,0].set_ylabel('Media E5-Large-v2')
        axes[1,0].legend()
        axes[1,0].grid(True, alpha=0.3)
        
        # 4. Matriz de correlación de métricas
        metrics_data = {
            'Ada Norma Media': [ada_properties['mean_norm']],
            'E5 Norma Media': [e5_properties['mean_norm']],
            'Similitud Coseno': [similarity_metrics['cosine_similarity_mean']],
            'Correlación Pearson': [similarity_metrics['pearson_correlation']],
            'Correlación Spearman': [similarity_metrics['spearman_correlation']],
            'Similitud Centroides': [similarity_metrics['centroid_similarity']]
        }
        
        # Crear tabla de resumen
        axes[1,1].axis('tight')
        axes[1,1].axis('off')
        
        table_data = [
            ['Métrica', 'Ada-002', 'E5-Large-v2'],
            ['Dimensiones', f"{ada_properties['dimensions']}", f"{e5_properties['dimensions']}"],
            ['Norma Media', f"{ada_properties['mean_norm']:.4f}", f"{e5_properties['mean_norm']:.4f}"],
            ['Valor Medio', f"{ada_properties['mean_value']:.6f}", f"{e5_properties['mean_value']:.6f}"],
            ['Desv. Estándar', f"{ada_properties['std_value']:.6f}", f"{e5_properties['std_value']:.6f}"],
            ['Sparsity (%)', f"{ada_properties['sparsity']*100:.2f}", f"{e5_properties['sparsity']*100:.2f}"],
            ['', '', ''],
            ['Similitud Coseno', f"{similarity_metrics['cosine_similarity_mean']:.4f}", '±' + f"{similarity_metrics['cosine_similarity_std']:.4f}"],
            ['Correlación Pearson', f"{similarity_metrics['pearson_correlation']:.4f}", ''],
            ['Correlación Spearman', f"{similarity_metrics['spearman_correlation']:.4f}", ''],
            ['Similitud Centroides', f"{similarity_metrics['centroid_similarity']:.4f}", '']
        ]
        
        table = axes[1,1].table(cellText=table_data, cellLoc='center', loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(9)
        table.scale(1.2, 1.5)
        
        # Colorear encabezados
        for i in range(len(table_data[0])):
            table[(0, i)].set_facecolor('#E8E8E8')
            table[(0, i)].set_text_props(weight='bold')
        
        axes[1,1].set_title('Resumen de Métricas Comparativas')
        
        plt.tight_layout()
        
        # Guardar visualización
        viz_path = f"{DRIVE_BASE}/comparison_visualization.png"
        plt.savefig(viz_path, dpi=300, bbox_inches='tight')
        plt.show()
        
        print(f"✅ Visualización guardada en: {viz_path}")
        
    except Exception as e:
        print(f"⚠️  Error generando visualizaciones: {e}")
        print("💡 Las visualizaciones son opcionales, el análisis numérico se completó correctamente")
        
else:
    print("⏭️  Visualizaciones omitidas (comparación no habilitada)")

In [ ]:
# Ejecutar análisis comparativo
if COMPARISON_ENABLED and ada_embeddings is not None:
    print("🚀 Iniciando análisis comparativo Ada-002 vs E5-Large-v2...")
    
    # Preparar embeddings E5 para comparación
    e5_embeddings = np.array([r['embedding'] for r in results_subset])
    
    print(f"📊 Configuración del análisis:")
    print(f"   Ada-002: {ada_embeddings.shape}")
    print(f"   E5-Large-v2: {e5_embeddings.shape}")
    
    # Análisis de propiedades individuales
    ada_properties = analyze_embedding_properties(ada_embeddings, "Ada-002")
    e5_properties = analyze_embedding_properties(e5_embeddings, "E5-Large-v2")
    
    # Métricas de similitud entre modelos
    similarity_metrics = calculate_similarity_metrics(ada_embeddings, e5_embeddings)
    
    print("\n📈 Resultados del análisis:")
    print("=" * 50)
    
    # Propiedades comparativas
    print(f"📊 PROPIEDADES DE EMBEDDINGS:")
    print(f"{'Métrica':<25} {'Ada-002':<15} {'E5-Large-v2':<15}")
    print("-" * 55)
    print(f"{'Dimensiones':<25} {ada_properties['dimensions']:<15} {e5_properties['dimensions']:<15}")
    print(f"{'Norma promedio':<25} {ada_properties['mean_norm']:<15.4f} {e5_properties['mean_norm']:<15.4f}")
    print(f"{'Valor promedio':<25} {ada_properties['mean_value']:<15.6f} {e5_properties['mean_value']:<15.6f}")
    print(f"{'Desv. estándar':<25} {ada_properties['std_value']:<15.6f} {e5_properties['std_value']:<15.6f}")
    print(f"{'Rango min-max':<25} [{ada_properties['min_value']:.3f}, {ada_properties['max_value']:.3f}] [{e5_properties['min_value']:.3f}, {e5_properties['max_value']:.3f}]\")"})
    print(f"{'Sparsity (%)':<25} {ada_properties['sparsity']*100:<15.2f} {e5_properties['sparsity']*100:<15.2f}")
    print(f"{'Memoria (MB)':<25} {ada_properties['memory_mb']:<15.1f} {e5_properties['memory_mb']:<15.1f}")
    
    print(f"\n🔗 SIMILITUD ENTRE MODELOS:")
    print("-" * 40)
    print(f"Similitud coseno promedio: {similarity_metrics['cosine_similarity_mean']:.4f} ± {similarity_metrics['cosine_similarity_std']:.4f}")
    print(f"Rango similitud coseno: [{similarity_metrics['cosine_similarity_min']:.4f}, {similarity_metrics['cosine_similarity_max']:.4f}]")
    print(f"Correlación Pearson: {similarity_metrics['pearson_correlation']:.4f}")
    print(f"Correlación Spearman: {similarity_metrics['spearman_correlation']:.4f}")
    print(f"Similitud centroides: {similarity_metrics['centroid_similarity']:.4f}")
    print(f"Correlación matrices similitud: {similarity_metrics['similarity_matrix_correlation']:.4f}")
    
    # Interpretación de resultados
    print(f"\n💡 INTERPRETACIÓN:")
    print("-" * 20)
    cos_sim_mean = similarity_metrics['cosine_similarity_mean']
    if cos_sim_mean > 0.8:
        print("✅ Alta concordancia entre modelos (similitud > 0.8)")
    elif cos_sim_mean > 0.6:
        print("⚠️  Concordancia moderada entre modelos (similitud 0.6-0.8)")
    else:
        print("❌ Baja concordancia entre modelos (similitud < 0.6)")
    
    pearson_corr = similarity_metrics['pearson_correlation']
    if pearson_corr > 0.7:
        print("✅ Fuerte correlación lineal entre espacios de embeddings")
    elif pearson_corr > 0.4:
        print("⚠️  Correlación moderada entre espacios de embeddings")
    else:
        print("❌ Correlación débil entre espacios de embeddings")
        
else:
    print("⏭️  Análisis comparativo omitido (no habilitado o sin datos Ada-002)")

In [ ]:
# Instalar dependencias adicionales para comparación
if COMPARISON_ENABLED:
    !pip install scikit-learn scipy matplotlib seaborn

In [ ]:
# Funciones de comparación y métricas
def calculate_similarity_metrics(embeddings1, embeddings2, sample_size=1000):
    """Calcular métricas de similitud entre dos conjuntos de embeddings"""
    from sklearn.metrics.pairwise import cosine_similarity
    from scipy.stats import pearsonr, spearmanr
    
    print(f"🔬 Calculando métricas de similitud...")
    
    # Usar muestra si el dataset es muy grande
    if len(embeddings1) > sample_size:
        indices = np.random.choice(len(embeddings1), sample_size, replace=False)
        emb1_sample = embeddings1[indices]
        emb2_sample = embeddings2[indices]
        print(f"📊 Usando muestra de {sample_size:,} documentos para métricas")
    else:
        emb1_sample = embeddings1
        emb2_sample = embeddings2
        print(f"📊 Usando todos los {len(embeddings1):,} documentos")
    
    # Normalizar embeddings
    from sklearn.preprocessing import normalize
    emb1_norm = normalize(emb1_sample, norm='l2')
    emb2_norm = normalize(emb2_sample, norm='l2')
    
    # Similitud coseno promedio entre embeddings correspondientes
    cos_similarities = []
    for i in range(len(emb1_norm)):
        cos_sim = np.dot(emb1_norm[i], emb2_norm[i])
        cos_similarities.append(cos_sim)
    
    cos_similarities = np.array(cos_similarities)
    
    # Correlaciones
    # Flatten embeddings para correlación global
    emb1_flat = emb1_sample.flatten()
    emb2_flat = emb2_sample.flatten()
    
    pearson_corr, _ = pearsonr(emb1_flat, emb2_flat)
    spearman_corr, _ = spearmanr(emb1_flat, emb2_flat)
    
    # Similitud centroide
    centroid1 = np.mean(emb1_norm, axis=0)
    centroid2 = np.mean(emb2_norm, axis=0)
    centroid_similarity = np.dot(centroid1, centroid2)
    
    # Matriz de similitud inter-modelo (muestra pequeña)
    sample_indices = np.random.choice(len(emb1_norm), min(100, len(emb1_norm)), replace=False)
    sim_matrix_ada = cosine_similarity(emb1_norm[sample_indices])
    sim_matrix_e5 = cosine_similarity(emb2_norm[sample_indices])
    
    # Correlación de matrices de similitud
    matrix_correlation, _ = pearsonr(sim_matrix_ada.flatten(), sim_matrix_e5.flatten())
    
    return {
        'cosine_similarity_mean': float(np.mean(cos_similarities)),
        'cosine_similarity_std': float(np.std(cos_similarities)),
        'cosine_similarity_min': float(np.min(cos_similarities)),
        'cosine_similarity_max': float(np.max(cos_similarities)),
        'pearson_correlation': float(pearson_corr),
        'spearman_correlation': float(spearman_corr),
        'centroid_similarity': float(centroid_similarity),
        'similarity_matrix_correlation': float(matrix_correlation),
        'sample_size': len(emb1_sample)
    }

def analyze_embedding_properties(embeddings, name):
    """Analizar propiedades de un conjunto de embeddings"""
    print(f"🔍 Analizando propiedades de {name}...")
    
    # Estadísticas básicas
    emb_array = np.array(embeddings)
    
    properties = {
        'model_name': name,
        'total_documents': len(emb_array),
        'dimensions': emb_array.shape[1],
        'mean_norm': float(np.mean(np.linalg.norm(emb_array, axis=1))),
        'std_norm': float(np.std(np.linalg.norm(emb_array, axis=1))),
        'mean_value': float(np.mean(emb_array)),
        'std_value': float(np.std(emb_array)),
        'min_value': float(np.min(emb_array)),
        'max_value': float(np.max(emb_array)),
        'sparsity': float(np.mean(emb_array == 0.0)),
        'memory_mb': float(emb_array.nbytes / 1024 / 1024)
    }
    
    # Análisis de distribución por dimensión (muestra)
    sample_dims = min(10, emb_array.shape[1])
    dim_stats = []
    for i in range(sample_dims):
        dim_values = emb_array[:, i]
        dim_stats.append({
            'dimension': i,
            'mean': float(np.mean(dim_values)),
            'std': float(np.std(dim_values)),
            'min': float(np.min(dim_values)),
            'max': float(np.max(dim_values))
        })
    
    properties['dimension_sample_stats'] = dim_stats
    
    return properties

print("✅ Funciones de comparación definidas")

In [ ]:
# Cargar embeddings Ada-002 para comparación
ada_embeddings = None
ada_documents = None

if COMPARISON_ENABLED:
    try:
        print("📥 Cargando embeddings Ada-002...")
        with open(ADA_EMBEDDINGS_FILE, 'r', encoding='utf-8') as f:
            ada_data = json.load(f)
        
        ada_embeddings = np.array(ada_data['embeddings'])
        ada_documents = ada_data['documents']
        
        print(f"✅ Cargados {len(ada_embeddings):,} embeddings Ada-002")
        print(f"   Dimensiones Ada: {ada_embeddings.shape[1]}")
        print(f"   Documentos: {len(ada_documents):,}")
        
        # Verificar que coincidan los documentos
        if len(ada_documents) != len(results):
            print(f"⚠️  Número de documentos diferentes:")
            print(f"   Ada: {len(ada_documents):,}")
            print(f"   E5: {len(results):,}")
            print("💡 Se usará el conjunto común para comparación")
            
            min_docs = min(len(ada_documents), len(results))
            ada_embeddings = ada_embeddings[:min_docs]
            ada_documents = ada_documents[:min_docs]
            results_subset = results[:min_docs]
        else:
            results_subset = results
            
        print(f"📊 Conjunto final para comparación: {len(ada_embeddings):,} documentos")
        
    except Exception as e:
        print(f"❌ Error cargando embeddings Ada: {e}")
        COMPARISON_ENABLED = False

In [ ]:
# Configuración para comparación
COMPARISON_ENABLED = True  # Cambiar a False si no quieres comparar
ADA_EMBEDDINGS_FILE = f"{DRIVE_BASE}/ada_embeddings.json"  # Archivo con embeddings Ada-002

print(f"🔍 Comparación de modelos: {'Habilitada' if COMPARISON_ENABLED else 'Deshabilitada'}")

if COMPARISON_ENABLED:
    print(f"📂 Buscando embeddings Ada-002 en: {ADA_EMBEDDINGS_FILE}")
    
    try:
        import os
        if os.path.exists(ADA_EMBEDDINGS_FILE):
            print("✅ Archivo de embeddings Ada-002 encontrado")
        else:
            print("⚠️  Archivo de embeddings Ada-002 no encontrado")
            print("💡 Para habilitar comparación, sube un archivo con embeddings Ada-002")
            COMPARISON_ENABLED = False
    except Exception as e:
        print(f"❌ Error verificando archivo Ada: {e}")
        COMPARISON_ENABLED = False

# E5-Large-v2 GPU Embedding Generation

Este notebook procesa documentos con E5-Large-v2 usando GPU de Colab para máxima velocidad.

**Pasos:**
1. Subir archivo exportado de ChromaDB a Google Drive
2. Montar Google Drive en Colab
3. Procesar con GPU T4/V100
4. Descargar resultado para importar de vuelta

**Ventajas:**
- 🚀 **10-50x más rápido** que procesamiento local
- 🆓 **Completamente gratis** con GPU T4
- 💾 **Sin limitaciones de memoria** local
- ⚡ **Procesamiento en lotes optimizado**

## 1. Configuración del entorno

In [ ]:
# Verificar GPU disponible
!nvidia-smi

import torch
print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Instalar dependencias necesarias
!pip install sentence-transformers pandas pyarrow tqdm

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
import json
import time
from datetime import datetime
import gc
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías importadas correctamente")

## 2. Montar Google Drive y cargar datos

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Configurar rutas
DRIVE_BASE = "/content/drive/MyDrive/ChromaDB_Export"
INPUT_FILE = f"{DRIVE_BASE}/docs_ada_export.parquet"  # o .json
OUTPUT_FILE = f"{DRIVE_BASE}/docs_e5large_processed.parquet"
CHECKPOINT_FILE = f"{DRIVE_BASE}/e5_colab_checkpoint.json"

print(f"📁 Rutas configuradas:")
print(f"   Input: {INPUT_FILE}")
print(f"   Output: {OUTPUT_FILE}")
print(f"   Checkpoint: {CHECKPOINT_FILE}")

In [ ]:
# Cargar datos exportados
try:
    # Intentar cargar Parquet primero (más eficiente)
    if INPUT_FILE.endswith('.parquet'):
        df = pd.read_parquet(INPUT_FILE)
        print(f"✅ Cargado archivo Parquet: {len(df):,} documentos")
    else:
        # Cargar JSON como fallback
        with open(INPUT_FILE, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # Convertir a DataFrame
        records = []
        for i, (doc, meta) in enumerate(zip(data['documents'], data['metadatas'])):
            record = {'document': doc, 'id': f"doc_{i}"}
            if meta:
                record.update(meta)
            records.append(record)
        
        df = pd.DataFrame(records)
        print(f"✅ Cargado archivo JSON: {len(df):,} documentos")
    
    # Mostrar información del dataset
    print(f"\n📊 Información del dataset:")
    print(f"   Filas: {len(df):,}")
    print(f"   Columnas: {list(df.columns)}")
    print(f"   Memoria: {df.memory_usage(deep=True).sum() / 1e6:.1f} MB")
    
    # Mostrar muestra
    print(f"\n📋 Muestra de datos:")
    print(df.head(2).to_string())
    
except Exception as e:
    print(f"❌ Error cargando datos: {e}")
    print("\n💡 Asegúrate de que el archivo exportado esté en Google Drive en la ruta correcta")

## 3. Cargar modelo E5-Large-v2 optimizado para GPU

In [ ]:
# Cargar modelo E5-Large-v2 con optimizaciones GPU
print("📥 Cargando modelo E5-Large-v2...")
start_time = time.time()

model = SentenceTransformer('intfloat/e5-large-v2')

# Optimizar para GPU si está disponible
if torch.cuda.is_available():
    model = model.to('cuda')
    print(f"✅ Modelo cargado en GPU en {time.time() - start_time:.1f}s")
    
    # Configurar para máximo rendimiento
    torch.backends.cudnn.benchmark = True
    print("⚡ Optimizaciones GPU activadas")
else:
    print(f"⚠️  Modelo cargado en CPU en {time.time() - start_time:.1f}s")
    print("💡 Para mejor rendimiento, activa GPU en Runtime > Change runtime type")

print(f"📏 Dimensiones del modelo: {model.get_sentence_embedding_dimension()}")

## 4. Funciones de procesamiento optimizadas

In [ ]:
def preprocess_for_e5(text: str) -> str:
    """Preprocesar texto para E5-Large con optimizaciones"""
    if not text or not text.strip():
        return "passage: empty document"
    
    # Limpiar texto
    text = text.strip()
    
    # Truncar inteligentemente para E5 (óptimo ~512 tokens)
    if len(text) > 2000:  # ~1500 tokens aprox
        # Mantener inicio y final para preservar contexto
        text = text[:1000] + " ... " + text[-1000:]
    
    # Prefijo requerido por E5
    return f"passage: {text}"

def save_checkpoint(processed_count: int, total_count: int, results: list):
    """Guardar checkpoint del progreso"""
    checkpoint_data = {
        "timestamp": datetime.now().isoformat(),
        "processed_count": processed_count,
        "total_count": total_count,
        "progress_percent": (processed_count / total_count * 100) if total_count > 0 else 0,
        "results_count": len(results)
    }
    
    with open(CHECKPOINT_FILE, 'w') as f:
        json.dump(checkpoint_data, f, indent=2)
    
    print(f"💾 Checkpoint guardado: {processed_count:,}/{total_count:,} ({checkpoint_data['progress_percent']:.1f}%)")

def load_checkpoint():
    """Cargar checkpoint si existe"""
    try:
        with open(CHECKPOINT_FILE, 'r') as f:
            checkpoint = json.load(f)
        print(f"📂 Checkpoint encontrado: {checkpoint['processed_count']:,}/{checkpoint['total_count']:,} procesados")
        return checkpoint['processed_count']
    except FileNotFoundError:
        print("🆕 Iniciando procesamiento desde cero")
        return 0
    except Exception as e:
        print(f"⚠️  Error cargando checkpoint: {e}")
        return 0

print("✅ Funciones de procesamiento definidas")

## 5. Procesamiento principal con GPU acelerado

In [ ]:
# Configuración de procesamiento
BATCH_SIZE = 64 if torch.cuda.is_available() else 16  # Más grande para GPU
CHECKPOINT_FREQUENCY = 500  # Guardar cada 500 documentos

print(f"⚙️  Configuración:")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Checkpoint cada: {CHECKPOINT_FREQUENCY} docs")
print(f"   Total documentos: {len(df):,}")

# Cargar checkpoint si existe
start_index = load_checkpoint()

if start_index >= len(df):
    print("✅ ¡Procesamiento ya completado!")
else:
    print(f"🚀 Iniciando procesamiento desde índice {start_index:,}")

In [ ]:
# Procesamiento principal
results = []
total_docs = len(df)
start_time = time.time()

# Filtrar documentos válidos desde el índice de inicio
remaining_df = df.iloc[start_index:].copy()
valid_docs = remaining_df[remaining_df['document'].notna() & (remaining_df['document'].str.len() > 0)]

print(f"📊 Documentos válidos para procesar: {len(valid_docs):,}")
print(f"📊 Documentos saltados (vacíos): {len(remaining_df) - len(valid_docs):,}")

if len(valid_docs) == 0:
    print("⚠️  No hay documentos válidos para procesar")
else:
    print("\n🔥 Iniciando procesamiento con GPU...")
    
    try:
        # Procesar en lotes
        for i in tqdm(range(0, len(valid_docs), BATCH_SIZE), desc="Procesando lotes"):
            batch_df = valid_docs.iloc[i:i + BATCH_SIZE]
            
            # Preprocesar textos para E5
            texts = [preprocess_for_e5(doc) for doc in batch_df['document']]
            
            # Generar embeddings con GPU
            batch_start = time.time()
            embeddings = model.encode(
                texts,
                batch_size=BATCH_SIZE,
                show_progress_bar=False,
                convert_to_numpy=True,
                normalize_embeddings=False  # E5 no requiere normalización
            )
            batch_time = time.time() - batch_start
            
            # Guardar resultados
            for idx, (_, row) in enumerate(batch_df.iterrows()):
                result = {
                    'id': f"e5_doc_{start_index + i + idx}",
                    'document': row['document'],
                    'embedding': embeddings[idx].tolist(),
                    'metadata': {k: v for k, v in row.items() if k != 'document'}
                }
                results.append(result)
            
            # Estadísticas de rendimiento
            processed_count = start_index + i + len(batch_df)
            docs_per_sec = len(batch_df) / batch_time
            
            if (i + BATCH_SIZE) % (CHECKPOINT_FREQUENCY // BATCH_SIZE * BATCH_SIZE) == 0 or i + BATCH_SIZE >= len(valid_docs):
                elapsed_time = time.time() - start_time
                total_processed = len(results)
                overall_speed = total_processed / elapsed_time
                eta_seconds = (total_docs - processed_count) / overall_speed if overall_speed > 0 else 0
                
                print(f"\n📊 Progreso:")
                print(f"   Procesados: {processed_count:,}/{total_docs:,} ({processed_count/total_docs*100:.1f}%)")
                print(f"   Velocidad lote: {docs_per_sec:.1f} docs/seg")
                print(f"   Velocidad promedio: {overall_speed:.1f} docs/seg")
                print(f"   Tiempo transcurrido: {elapsed_time/60:.1f} min")
                print(f"   ETA: {eta_seconds/60:.1f} min")
                
                # Guardar checkpoint
                save_checkpoint(processed_count, total_docs, results)
                
                # Limpiar memoria GPU
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                gc.collect()
    
    except Exception as e:
        print(f"❌ Error durante procesamiento: {e}")
        print("💾 Guardando progreso parcial...")
        save_checkpoint(len(results), total_docs, results)
        raise
    
    # Resumen final
    total_time = time.time() - start_time
    print(f"\n🎉 ¡Procesamiento completado!")
    print(f"   Total procesado: {len(results):,} documentos")
    print(f"   Tiempo total: {total_time/60:.1f} minutos")
    print(f"   Velocidad promedio: {len(results)/total_time:.1f} docs/seg")
    print(f"   Dimensiones embeddings: {len(results[0]['embedding'])} (debe ser 1024)")

## 6. Guardar resultados finales

In [ ]:
# Convertir resultados a formato final
if results:
    print("💾 Preparando datos finales...")
    
    final_data = {
        'export_info': {
            'timestamp': datetime.now().isoformat(),
            'model': 'intfloat/e5-large-v2',
            'dimensions': 1024,
            'total_documents': len(results),
            'processing_time_minutes': (time.time() - start_time) / 60,
            'colab_gpu': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'
        },
        'documents': [r['document'] for r in results],
        'embeddings': [r['embedding'] for r in results],
        'metadatas': [r['metadata'] for r in results],
        'ids': [r['id'] for r in results]
    }
    
    # Guardar como JSON (compatible con el script de importación)
    json_output = OUTPUT_FILE.replace('.parquet', '.json')
    print(f"💾 Guardando resultados en: {json_output}")
    
    with open(json_output, 'w', encoding='utf-8') as f:
        json.dump(final_data, f, ensure_ascii=False, indent=2)
    
    # También guardar como Parquet para eficiencia
    try:
        results_df = pd.DataFrame([
            {
                'id': r['id'],
                'document': r['document'],
                'embedding': r['embedding'],
                **r['metadata']
            } for r in results
        ])
        
        results_df.to_parquet(OUTPUT_FILE, index=False)
        print(f"✅ Guardado adicional en Parquet: {OUTPUT_FILE}")
    except Exception as e:
        print(f"⚠️  No se pudo guardar en Parquet: {e}")
    
    # Verificar archivos guardados
    import os
    json_size = os.path.getsize(json_output) / (1024 * 1024)
    print(f"\n📊 Archivos generados:")
    print(f"   JSON: {json_output} ({json_size:.1f} MB)")
    if os.path.exists(OUTPUT_FILE):
        parquet_size = os.path.getsize(OUTPUT_FILE) / (1024 * 1024)
        print(f"   Parquet: {OUTPUT_FILE} ({parquet_size:.1f} MB)")
    
    print(f"\n✅ ¡Procesamiento completado exitosamente!")
    print(f"📁 Descarga los archivos desde Google Drive para importar de vuelta a ChromaDB")
    
else:
    print("❌ No hay resultados para guardar")

## 7. Verificación final y estadísticas

In [ ]:
# Verificación final
if results:
    print("🔍 Verificación final:")
    print(f"   Documentos procesados: {len(results):,}")
    print(f"   Dimensiones embeddings: {len(results[0]['embedding'])}")
    
    # Verificar que todas las dimensiones sean correctas
    dims_check = all(len(r['embedding']) == 1024 for r in results[:100])  # Verificar muestra
    print(f"   Dimensiones correctas: {'✅' if dims_check else '❌'}")
    
    # Estadísticas de embeddings
    sample_embedding = np.array(results[0]['embedding'])
    print(f"   Rango embeddings: [{sample_embedding.min():.3f}, {sample_embedding.max():.3f}]")
    print(f"   Media embedding: {sample_embedding.mean():.3f}")
    
    # Memoria GPU final
    if torch.cuda.is_available():
        memory_used = torch.cuda.memory_allocated() / 1e9
        memory_total = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"   Memoria GPU: {memory_used:.1f}/{memory_total:.1f} GB")
    
    print("\n🎉 ¡Todo listo! Los archivos están listos para descargar e importar.")
    print("\n📋 Próximos pasos:")
    print("1. Descargar el archivo JSON/Parquet desde Google Drive")
    print("2. Ejecutar el script de importación en tu máquina local")
    print("3. Verificar la importación con verify_e5_migration.py")
    print("4. ¡Disfrutar de E5-Large-v2 súper rápido!")
else:
    print("❌ No se generaron resultados válidos")

## 8. Comparación con embeddings Ada-002 (Opcional)

Si tienes embeddings Ada-002 existentes, puedes ejecutar esta sección para generar métricas de comparación.